In [1]:
import fastscape_litho as fstl
import fastscape
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import zarr
%matplotlib widget
%load_ext xsimlab.ipython

mymod = fstl.sediment_model_label3D_SPL.update_processes({'flow': fastscape.processes.SingleFlowRouter, "TA": fstl.Quicksn})# just SPL
# mymod = fstl.sediment_model_label3D # full model

In [2]:
ny,nx = 200,200
ly,lx = 2e4,2e4
m,n = 0.45,1.11
flowexp = 7
Ksoil = 5e-4

nz = 300
dz = 60

G_r = 0.01
G_s = 0.1

Up = np.full( (ny,nx) , 1e-3 )
labmat = np.zeros((ny,nx,nz), dtype = np.int8)
# labmat[50:150,50:150,30:] = 1

Krs = np.array([2e-4,1e-4, 0.6e-4, 1e-4]) * 0.13
Kdrs = np.array([2e-2,1e-2, 0.6e-2,1e-2]) * 0.13
Kdss = Kdrs * 1.3 * 0.2

dt = 1e4
time = np.arange(0,2e7,dt)
otime = time[::5]


In [3]:
xz = np.zeros((nz,nx))
a = 0.05
b = 100
c = 130
d = 80
thisckness = 60
for i in range(nx):
    tz = d * np.cos(a * (i + b)) + c
    tz = round(tz)
    xz[tz:,i] = 1
    tz = d * np.cos(a * (i + b)) + c + thisckness
    tz = round(tz)
    xz[tz:,i] = 2




for i in range(nx):
    labmat[:,i,:] = np.rot90(xz)
    
fig,ax = plt.subplots(figsize = (6,6))
ax.imshow(labmat[:,50,:])    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# %create_setup mymod
import xsimlab as xs
### IMPORTANT!!!! Adding that here to get a test dummy homogeneous 3D block with the default value
labmat = np.zeros((ny,nx,nz), dtype = np.int8) - 1

# SPL model
ds_in = xs.create_setup(
    model=mymod,
        clocks={
        'time':time,
        'otime':otime,
    },
    master_clock= 'time',
    input_vars={
        'grid__shape': [ny,nx],
        'grid__length': [ly,lx],
        'boundary__status': ['looped', 'looped', 'fixed_value', 'fixed_value'],
        'uplift__rate': Up,
        'init_topography__seed': 21,
#         'flow__slope_exp': flowexp,
        'spl__area_exp': m,
        'spl__slope_exp': n,
        'label__dz': dz,
        'label__origin_z': 0,
        'label__labelmatrix': labmat,
        'label__k_lab': Krs,
        'TA__theta_chi': m/n,
        'TA__A_0_chi': 1,
        'TA__minAcc': 1e5
    },
    output_vars={
        'topography__elevation': 'otime',
        'drainage__area' : 'otime',
        'label__indices': 'otime',
        'TA__chi': 'otime',
        'TA__ksn': 'otime',
        'erosion__rate' : 'otime'
    }
)

#Full model
# %create_setup mymod


# ds_in = xs.create_setup(
#     model=mymod,
#     clocks={
#         'time':time,
#         'otime':otime,
#     },
#     input_vars={
#         'grid__shape': [ny,nx],
#         'grid__length': [ly,lx],
#         'boundary__status': ['looped', 'looped', 'fixed_value', 'fixed_value'],
#         'uplift__rate': Up,
#         'init_topography__seed': 21,
#         'flow__slope_exp': flowexp,
#         'spl__area_exp': n,
#         'spl__slope_exp': m,
#         'spl__k_coef_soil': Ksoil,
#         'spl__g_coef_bedrock': 0.01,
#         'spl__g_coef_soil': 0.1,
#         'label__dz': dz,
#         'label__origin_z': 0,
#         'label__labelmatrix': labmat,
#         'label__Kr_lab': Krs,
#         'label__Kdr_lab': Kdrs,
#         'label__Kds_lab': Kdss,
#         'label__infiltration_lab': Krs,
#     },
#     master_clock = 'time',
#     output_vars={
#         'topography__elevation': 'otime',
#         'drainage__area' : 'otime',
#         'label__indices': 'otime'
#     }
# )



In [ ]:
with mymod, xs.monitoring.ProgressBar():
#     out_ds = ds_in.xsimlab.run(store=zarr.TempStore())
    out_ds = ds_in.xsimlab.run()

             0% | initialize 

In [ ]:
from ipyfastscape import TopoViz3d
app = TopoViz3d(out_ds, canvas_height=600,time_dim='otime')
app.show()

In [ ]:
ksn_SS = []

for t in out_ds.otime.values:
    print(t, end = '|')
    ksn_SS.append( np.power( ( Up / Krs[out_ds["label__indices"].sel({"otime":t}).values.astype(np.int16)] ) , (1/n)) )



In [ ]:
import xarray as xr
out_ds["ksn_SS"] = xr.DataArray(ksn_SS, dims = ('otime','y','x'))
out_ds["delta_ksn"] = out_ds["TA__ksn"] - out_ds["ksn_SS"]

In [ ]:
X = out_ds['otime'].values
Y = []
Y_1st = []
Y_3rd = []
for i in range(out_ds.otime.values.shape[0]):
    Y.append(np.median(out_ds["delta_ksn"].values[i][out_ds["TA__chi"].values[i]>0]))
    Y_1st.append(np.percentile(out_ds["delta_ksn"].values[i][out_ds["TA__chi"].values[i]>0], 25))
    Y_3rd.append(np.percentile(out_ds["delta_ksn"].values[i][out_ds["TA__chi"].values[i]>0], 75))
    
fig, ax = plt.subplots(figsize = (7,6))
ax.plot(X,Y)
